In [ ]:
import pandas
from google.colab import files
uploaded = files.upload()
train_data = pandas.read_csv(next(iter(uploaded)))
uploaded = files.upload()
test_data = pandas.read_csv(next(iter(uploaded)))
print("Train dataset:")
print(train_data.head())
print("\nTest dataset:")
print(test_data.head())


Saving train.csv to train (6).csv


Saving test.csv to test (6).csv
Train dataset:
   Unnamed: 0      id  Gender      Customer Type  Age   Type of Travel  \
0           0   70172    Male     Loyal Customer   13  Personal Travel   
1           1    5047    Male  disloyal Customer   25  Business travel   
2           2  110028  Female     Loyal Customer   26  Business travel   
3           3   24026  Female     Loyal Customer   25  Business travel   
4           4  119299    Male     Loyal Customer   61  Business travel   

      Class  Flight Distance  Inflight wifi service  \
0  Eco Plus              460                      3   
1  Business              235                      3   
2  Business             1142                      2   
3  Business              562                      2   
4  Business              214                      3   

   Departure/Arrival time convenient  ...  Inflight entertainment  \
0                                  4  ...                       5   
1                                  2  .

In [ ]:
train_data.dropna(inplace=True)
train_data.dropna(axis=1, how='all', inplace=True)
test_data.dropna(inplace=True)
test_data.dropna(axis=1, how='all', inplace=True)


In [ ]:
#Database schema
import sqlite3
import pandas
from tabulate import tabulate
data = {
    "PassengerID": [70172, 5047],
    "Gender": ["Male", "Male"],
    "CustomerType": ["Loyal Customer", "Disloyal Customer"],
    "Age": [13, 25],
    "TypeOfTravel": ["Personal Travel", "Business travel"],
    "Class": ["Eco Plus", "Business"],
    "FlightDistance": [460, 235],
    "InflightWifiService": [3, 3],
    "DepartureArrivalTimeConvenient": [4, 2],
    "EaseOfOnlineBooking": [3, 3],
    "GateLocation": [1, 3],
    "FoodAndDrink": [5, 1],
    "OnlineBoarding": [3, 3],
    "SeatComfort": [5, 1],
    "InflightEntertainment": [4, 5],
    "OnboardService": [3, 3],
    "LegRoomService": [4, 1],
    "BaggageHandling": [4, 4],
    "CheckinService": [5, 1],
    "InflightService": [5, 4],
    "Cleanliness": [5, 1],
    "DepartureDelayMinutes": [25, 6],
    "ArrivalDelayMinutes": [18, None],
    "Satisfaction": ["neutral or dissatisfied", "neutral or dissatisfied"]
}
df = pandas.DataFrame(data)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
create_table_sql = """
CREATE TABLE Passenger (
    PassengerID INT PRIMARY KEY,
    Gender VARCHAR(10),
    CustomerType VARCHAR(20),
    Age INT,
    TypeOfTravel VARCHAR(20),
    Class VARCHAR(20),
    FlightDistance INT,
    InflightWifiService INT,
    DepartureArrivalTimeConvenient INT,
    EaseOfOnlineBooking INT,
    GateLocation INT,
    FoodAndDrink INT,
    OnlineBoarding INT,
    SeatComfort INT,
    InflightEntertainment INT,
    OnboardService INT,
    LegRoomService INT,
    BaggageHandling INT,
    CheckinService INT,
    InflightService INT,
    Cleanliness INT,
    DepartureDelayMinutes INT,
    ArrivalDelayMinutes INT,
    Satisfaction VARCHAR(20)
);
"""
cursor.execute(create_table_sql)
df.to_sql('Passenger', conn, if_exists='append', index=False)
conn.commit()
cursor.execute("PRAGMA table_info('Passenger')")
schema_rows = cursor.fetchall()
schema_table = tabulate(schema_rows, headers=["Column Name", "Data Type"], tablefmt='grid')
print("Table Schema:")
print(schema_table)
cursor.execute("SELECT * FROM Passenger")
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
print("\nTable Data:")
data_table = tabulate(rows, headers=columns, tablefmt='grid')
print(data_table)
cursor.close()
conn.close()


Table Schema:
+----+--------------------------------+-------------+----+---------------+-------------+
|    |                                |             |    | Column Name   |   Data Type |
+====+================================+=============+====+===============+=============+
|  0 | PassengerID                    | INT         |  0 |               |           1 |
+----+--------------------------------+-------------+----+---------------+-------------+
|  1 | Gender                         | VARCHAR(10) |  0 |               |           0 |
+----+--------------------------------+-------------+----+---------------+-------------+
|  2 | CustomerType                   | VARCHAR(20) |  0 |               |           0 |
+----+--------------------------------+-------------+----+---------------+-------------+
|  3 | Age                            | INT         |  0 |               |           0 |
+----+--------------------------------+-------------+----+---------------+-------------+
|  4 | 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from tabulate import tabulate
X_train = train_data.drop(columns=["satisfaction"])
y_train = train_data["satisfaction"]
X_test = test_data.drop(columns=["satisfaction"])
y_test = test_data["satisfaction"]
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_imputed, y_train)
y_pred = clf.predict(X_test_imputed)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
feature_importances = list(zip(X_train.columns, clf.feature_importances_))
print("\nFeature Importances:")
print(tabulate(feature_importances, headers=["Feature", "Importance"], tablefmt="grid"))


Accuracy: 0.9444487218971358

Feature Importances:
+-----------------------------------+--------------+
| Feature                           |   Importance |
+===================================+==============+
| Unnamed: 0                        |   0.0137544  |
+-----------------------------------+--------------+
| id                                |   0.0268351  |
+-----------------------------------+--------------+
| Gender                            |   0.00182551 |
+-----------------------------------+--------------+
| Customer Type                     |   0.0406672  |
+-----------------------------------+--------------+
| Age                               |   0.017904   |
+-----------------------------------+--------------+
| Type of Travel                    |   0.146591   |
+-----------------------------------+--------------+
| Class                             |   0.0211803  |
+-----------------------------------+--------------+
| Flight Distance                   |   0.015170

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
train_data.dropna(subset=["satisfaction"], inplace=True)
train_data.drop(train_data[train_data["satisfaction"] == ""].index, inplace=True)
train_data["satisfaction"] = train_data["satisfaction"].map({"neutral or dissatisfied": 0, "satisfied": 1})
train_data["Gender"] = train_data["Gender"].map({"Male": 0, "Female": 1})
train_data["Customer Type"] = train_data["Customer Type"].map({"disloyal Customer": 0, "Loyal Customer": 1})
train_data["Type of Travel"] = train_data["Type of Travel"].map({"Personal Travel": 0, "Business travel": 1})
train_data["Class"] = train_data["Class"].map({"Eco Plus": 0, "Business": 1, "Eco": 2})
X_train = train_data.drop(columns=["satisfaction"])
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_train_tensor = torch.tensor(X_train.astype('float32'))
y_train_tensor = torch.tensor(train_data["satisfaction"].values.astype('float32')).reshape(-1, 1)
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
input_size = X_train_tensor.shape[1]
model = Net(input_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
with torch.no_grad():
    y_pred_train = torch.sigmoid(model(X_train_tensor))
y_pred_train_binary = torch.round(y_pred_train)
y_train_np = y_train_tensor.detach().numpy()
y_pred_train_binary_np = y_pred_train_binary.detach().numpy()



/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Epoch [10/100], Loss: nan
Epoch [20/100], Loss: nan
Epoch [30/100], Loss: nan
Epoch [40/100], Loss: nan
Epoch [50/100], Loss: nan
Epoch [60/100], Loss: nan
Epoch [70/100], Loss: nan
Epoch [80/100], Loss: nan
Epoch [90/100], Loss: nan
Epoch [100/100], Loss: nan


In [ ]:
# Define SQL statements to create tables
sql_statements = [
    """
    CREATE TABLE Passenger (
        PassengerID INT PRIMARY KEY,
        Gender VARCHAR(10),
        CustomerType VARCHAR(20),
        Age INT,
        TypeOfTravel VARCHAR(20),
        Class VARCHAR(20),
        FlightDistance INT,
        -- Add other attributes as needed
    )
    """,
    """
    CREATE TABLE Flight (
        FlightID INT PRIMARY KEY,
        DepartureTime DATETIME,
        ArrivalTime DATETIME,
        FlightDistance INT,
        -- Add other attributes as needed
    )
    """,
    """
    CREATE TABLE Satisfaction (
        SatisfactionID INT PRIMARY KEY,
        SatisfactionLevel VARCHAR(20)
    )
    """,
    """
    CREATE TABLE PassengerFlight (
        PassengerID INT,
        FlightID INT,
        FOREIGN KEY (PassengerID) REFERENCES Passenger(PassengerID),
        FOREIGN KEY (FlightID) REFERENCES Flight(FlightID),
        PRIMARY KEY (PassengerID, FlightID)
    )
    """,
    """
    ALTER TABLE Passenger
    ADD COLUMN SatisfactionID INT,
    FOREIGN KEY (SatisfactionID) REFERENCES Satisfaction(SatisfactionID)
    """
]

# Execute SQL statements
for statement in sql_statements:
    print(statement)
    # Execute the statement using your database connection



    CREATE TABLE Passenger (
        PassengerID INT PRIMARY KEY,
        Gender VARCHAR(10),
        CustomerType VARCHAR(20),
        Age INT,
        TypeOfTravel VARCHAR(20),
        Class VARCHAR(20),
        FlightDistance INT,
        -- Add other attributes as needed
    )
    

    CREATE TABLE Flight (
        FlightID INT PRIMARY KEY,
        DepartureTime DATETIME,
        ArrivalTime DATETIME,
        FlightDistance INT,
        -- Add other attributes as needed
    )
    

    CREATE TABLE Satisfaction (
        SatisfactionID INT PRIMARY KEY,
        SatisfactionLevel VARCHAR(20)
    )
    

    CREATE TABLE PassengerFlight (
        PassengerID INT,
        FlightID INT,
        FOREIGN KEY (PassengerID) REFERENCES Passenger(PassengerID),
        FOREIGN KEY (FlightID) REFERENCES Flight(FlightID),
        PRIMARY KEY (PassengerID, FlightID)
    )
    

    ALTER TABLE Passenger
    ADD COLUMN SatisfactionID INT,
    FOREIGN KEY (SatisfactionID) REFERENCES Satisfaction(